<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Training Binary Neural Networks by Integer Linear Programming</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Run the following script whenever running this script on a Google Colab.

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

**REMARK:** If working on your personal computer, you might want to install [Gurobi with an academic license](https://www.gurobi.com/academia/academic-program-and-licenses/). Gurobi is a commercial solver that is extremely fast in solving large ILP instances, much more faster than the free GLPK solver.

# 7. Training (Binary) Neural Networks by Integer Linear Programming
In this notebook, we show how to write an ILP model to train (binary) neural networks. We start by considering a basic perceptron, then we defined multilayer neural network, and, finally, we focus on binary neural networks. 
Notice that this type of approach will not likely scale to the size of the input, but it can help to reason about the optimal architecture of Neural Networks in small examples.

Let the pair $\mathcal{X} = (X, Y)$ be the input training dataset, where $X\in \mathbb{R}^{n \times m}$ and $Y\in \mathbb{R}^n$.
Each row $x_i \in \mathbb{R}^m$ of the matrix $X$ rapresent an input data point, which is mapped to the $i$-th label $y_i$.
The dataset is partinioned into a training set $\mathcal{T} \subset \mathcal{X}$ and a validat set $\mathcal{S} \subset \mathcal{T}$.

A basic perceptron is defined by the following parametric function $f : \mathbb{R}^{n \times m} \rightarrow \mathbb{R}^n$:

$$
    \hat{y}= f_W(X), \text{ where } X \in \mathbb{R}^{n\times m}, \hat{y} \in \mathbb{R^n}, W \in \mathbb{R^n}
$$

The parametric function is defined as:

$$
    f_W(X) = \text{sign}( X \cdot W ) 
$$

or componentwise:

$$
    f_W(x_i) = \text{sign}( x_i \cdot W ) 
$$

where the sign function is +1 for positive input, and -1 otherwise.

Using the training dataset $\mathcal{T}$, we are interested in finding the *best* parameters $W^*$:

$$
    W^* = \argmin_{W \in \mathbb{R}^n} || y - \hat{y} || = \argmin_{W \in \mathbb{R}^n} || y - f_W(X) ||, \quad \text{ with } (X,y) \in \mathcal{T}
$$

Later, we evaluate the perceproton computing the accuracy on the validation set $\mathcal{S}$:
$$
    \text{accuracy} = \frac{\sum || y - f_{W^*}(X) ||}{|\mathcal{S}|}, \quad \text{ with } (X,y) \in \mathcal{S}
$$

We show next how to model using Integer Linear Programming the problem of finding the best $W^*$.

## 7.1 Learning basic logical operators
From a didatic perspective, learning the three basic logical operators **and**, **or**, and **xor** is an excellent exercise. Let us start with the **and** operator, whose true table is given below:

| i | $x'_1(i)$ | $x'_2(i)$ | $y'(i)$ |
|--|----|----|---|
| 1 | 0 | 0 | 0 |
| 2 | 0 | 1 | 0 |
| 3 | 0 | 0 | 0 |
| 4 | 1 | 1 | 1 |

Moreover, we can reparametrize the data into the value -1,+1 (by using the transformation $x(i) = 2x'(i) - 1$ as follows.

| i | $x_1(i)$ | $x_2(i)$ | $y(i)$ |
|--|----|----|---|
| 1 | -1 | -1 | -1 |
| 2 | -1 | +1 | -1 |
| 3 | -1 | -1 | -1 |
| 4 | +1 | +1 | +1 |

We can then use the values of $x_1$ and $x_2$ to define the training matrix $X\in \mathbb{R}^{4 \times 2}$ and vector $y \in \mathbb{R}^4$.

To start, we will use the true table as both the training and validation dataset.

TODO: complete with the notes from last lab session.


In [ ]:
# AND function (with first element for bias)
Xand = [(1,-1,-1), (1,-1, 1), (1,1,-1), (1,1,1)]
Yand = [-1, -1, -1, 1]

# OR function (with first element for bias)
Xor = [(1,-1,-1), (1,-1, 1), (1,1,-1), (1,1,1)]
Yor = [-1, 1, 1, 1]

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
from pyomo.environ import maximize, minimize, Binary, RangeSet, ConstraintList, NonNegativeReals, Reals, Integers

def LogicalNN(Xs, Ys):
    # Main Pyomo model
    model = ConcreteModel()

    # TODO: complete the model
    # ...

    # Return weight of the final solution
    return []

For a possible solution, look at the following link: 

https://github.com/mathcoding/opt4ds/blob/master/scripts/NN_and.py

### 7.1.1 Noisy input data
To consider a more realistic setting, let us suppose that the input has a noise, that is, for instance:
$$
    y = f_W(X + \epsilon)
$$
where $\epsilon \in \mathbb{R}^{n \times m}$ is any kind of noise coming from an unknown distribution (e.g., a uniform, normal, or lognormal).

The following function can be used to add noise to the input.

In [ ]:
from numpy.random import normal
def AddNoise(X, mu=0.1):
    return list(map(lambda x: x+normal(0, mu), X))

In [ ]:
# To increase the size of the input dataset:
T = 10
Xs = T*[AddNoise(X, 0.1) for X in Xand]
Ys = T*Yand
print(Xs[:5], Ys[:5])

**EXERCISE 1:** Adapt the previous script to be trained over random input data.

## 7.2 The **xor** logical function
The simple perceptron is unable to correctly learn the **xor** function because it is not linearly separable. One possibility to overcome this limitation is to change the structure of the parametric function $f_W$.

For instance, we could use the following function, by adding a new vector of parameters U:
$$
    \hat{y} = f_{W,U} = \text{sign}(U \cdot \text{sign}(X W)) 
$$

Notice that now we have a *hidden* layer of unknows given by the inner $\text{sign}$ function. The inner product between the weight $U$ and the result of the sign function, introduce bilinear terms that must be carfully linearized, in order to use Integer Linear Programming to solve the training problem.

TODO: complete with the lab notes on the iPad.

In [ ]:
# XOR function
Xxor = [(1,-1,-1), (1,-1, 1), (1,1,-1), (1,1,1)]
Yxor = [-1, 1, 1, -1]

In [ ]:
def MLP(Xs, Ys, nh):
    # Main Pyomo model
    model = ConcreteModel()

    # TODO: complete the model
    # ...

    # Return weight of the final solution W, U
    return [], []

Then, you have to design the validation test.

**EXERCISE 2:** Modify your script in such a way that the weight belong only to the set of values $\{-1, 0, +1\}$. Note that a value of 0 is equivalent to remove the corresponding link (and simplify the network).

**EXERCISE 3:** Can you proof which is the smallest Binary Neural Network that can compute exactly the **xor** function? Hown many weights do you need in total?

## 7.3 Classification of MNIST digits
Starting from the solution used to model the **xor** function, you can build a parametric function whose parameters are fitted to classify images.

You can use the same structure used for the **xor** function, but by chaning the number of states in the hidden layer:
$$
    \hat{y} = f_{W,U} = \text{sign}(U \cdot \text{sign}(X W)) 
$$

Or you can propose any different type of solution.

You can download the dataset as follows.

In [ ]:
# Uncomment and exectue
# !wget https://mate.unipv.it/gualandi/opt4ds/all_three_four.csv

In [ ]:
# Uncomment and exectue
# !wget https://mate.unipv.it/gualandi/opt4ds/all_none_four.csv

## 7.3.1 Parsing the dataset
To parse the dataset you can use the following function

In [ ]:
import numpy as np
def Parse(filename):
    fh = open(filename, 'r')

    fh.readline()

    Xs, Ys = [], []
    for row in fh:
        line = row.replace('\n','').split(';')
        Ys.append(int(line[0]))
        Xs.append(list(map(int, line[1:])))

    return np.matrix(Xs), np.array(Ys)

In [ ]:
Xs, Ys = Parse('../data/train_three_four.csv')

In [ ]:
print('dimension of matrix X:', Xs.shape)

In [ ]:
print('dimension of y:', Ys.shape)

## 7.3.2 Plotting digits
To plot a digit you can use the following snippet.

In [ ]:
import matplotlib.pyplot as plt

def DrawDigit(A):
    plt.imshow(A.reshape((28,28)), cmap='binary')
    plt.show()

In [ ]:
DrawDigit(Xs[2])

## 7.4.3 Evaluating a NN
To evaluate the accuracy of a Binary NN you can run the following function.

In [ ]:

def AccuracyMLP(Xs, Ys, W, U):
    y_hat = np.sign( np.matmul( np.sign(np.matmul(Xs, W)), np.transpose(U)))

    n = len(Ys)
    return (np.sum(Ys == y_hat))/n*100

In [ ]:
# Number of internal states
nh = 2

# REMARK: the value of W and U should be the output of your neural network
W = np.ones( (784, nh) )
U = np.ones(nh)

# Evaluate accuracy (be careful of the matrix dimension)
acc = AccuracyMLP(Xs, Ys, W, U)
print('accuracy:', round(acc, 2))

## 7.4 BNN Classification Challenge
For this challenge, you have to design a binary neural network, that is a binary neural network where all weights are either +1 or -1, and that is able to solve a binary classification problem defined on pair of MNIST images.

For training, you will have two dataset, the first containing images of the digits 3 and 4, and the second, containing the images of digits 4 and 9. 

For the design phase, you should use a small number of input data points. Later, you can decide if having a *light* model that can take in input several data points, or a *heavy* model that can use only a few data points but is more general.

The evaluation will be carried over a hidden mixed dataset.

*REMARK*: You have to submit your solution by Thursday, 25th, 2023, sending an email containg your python solution script.

Partecipating to this (optional) challenge, you will get extra points for the final exam grade.